In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import datasets, transforms
import helper
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report,accuracy_score
from google.colab import files
from torchsummary import summary
# import onnx
# from onnx2pytorch import ConvertModel
from tqdm import trange
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


links to fine tuned models:


*   http://ptak.felk.cvut.cz/personal/sulcmila/models/LifeCLEF2018/

*   http://ptak.felk.cvut.cz/personal/sulcmila/models/LifeCLEF2019/

*   www.leafnet.pbarre.de 

*   https://www.kaggle.com/datasets/maksymshkliarevskyi/cassava-leaf-disease-models?select=EfNetB0_275_16.h5



In [3]:
#path_to_model = 'drive/MyDrive/trained_models/resnet18.onnx'
path_to_model = 'drive/MyDrive/trained_models/deepweeds_resnet50_pretrained'
feature_extract = False

In [4]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


# build model to map pretrained deepweeds model
model = models.resnet50()
set_parameter_requires_grad(model, feature_extract)
model.fc = nn.Linear(2048, 9)
model.load_state_dict(torch.load(path_to_model), strict=False)

# change output later to map csic data
model.fc = nn.Linear(2048, 4)


In [5]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
model = model.to(device)

In [7]:
# build dataloader
mean = torch.tensor([135.31470632, 124.53849418, 103.39646082])
std = torch.tensor([5.18153318, 4.14170719, 5.17011963])

# transform = transforms.Compose([transforms.Resize(255),
#                                 transforms.CenterCrop(224),
#                                 transforms.ToTensor(),
#                                 transforms.RandomRotation(degrees=360),
#                                 transforms.RandomHorizontalFlip(p=0.5),
#                                 transforms.ColorJitter(brightness=.5, hue=.3),
#                                 transforms.RandomInvert(),
#                                 transforms.Normalize(mean, std)])

transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])

balanced_path_data = 'drive/MyDrive/balanced_csic_data_64/training'


def train_val_dataset(dataset, val_split=0.1):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

# load the balanced dataset
balanced_dataset = datasets.ImageFolder(balanced_path_data, transform=transform)
balanced_datasets = train_val_dataset(balanced_dataset)


class_dict = balanced_dataset.class_to_idx

In [8]:
batch_size = 32
balanced_dataloaders_dict = {x: torch.utils.data.DataLoader(balanced_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train', 'val']}


In [9]:
test_path_data = 'drive/MyDrive/balanced_csic_data_64/testing'
testing_dataset = torchvision.datasets.ImageFolder(test_path_data, transform=transform)
batch_size = 10
testing_dataloader =torch.utils.data.DataLoader(testing_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
print(testing_dataset.class_to_idx)

{'Cyperus': 0, 'Negative': 1, 'Portulaca': 2, 'Solanum': 3}


In [10]:
# send model to GPU
if torch.cuda.is_available():
    model.cuda()

In [13]:
def getTestingMetrics(dataloader, model):
  # send model to GPU
  if torch.cuda.is_available():
    model.cuda()

  # with tqdm(testing_dataloader) as tepoch:
  lbs = []
  preds = []
  #   # Iterate over data.
  for inputs, labels in dataloader:
    test_inputs = inputs.to(device)
    #test_labels = labels.to(device)
    outputs = model(test_inputs)
    _, pred = torch.max(outputs, 1)
    lbs+=(list(labels.numpy()))
    preds+= list(pred.cpu().data.numpy())
  print('Overall testing accuracy:',accuracy_score(lbs, preds))
  print(classification_report(lbs, preds, target_names=class_dict.keys()))


In [14]:
getTestingMetrics(testing_dataloader, model)

Overall testing accuracy: 0.4858823529411765
              precision    recall  f1-score   support

     Cyperus       0.00      0.00      0.00       302
    Negative       0.50      0.96      0.65       425
   Portulaca       0.13      0.04      0.06        85
     Solanum       0.00      0.00      0.00        38

    accuracy                           0.49       850
   macro avg       0.16      0.25      0.18       850
weighted avg       0.26      0.49      0.33       850



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


tESTING OUT A MODEL DIRECTLY ON iMAEnET